In [9]:
import numpy as np
from pyomo.environ import *
model = ConcreteModel()
model.I = Set(initialize = range(5)) # 采用列表初始化技术参照决策单元个数的集合
model.K = Set(initialize = range(3)) # 采用列表初始化投入变量个数的集合
model.L = Set(initialize = range(2)) # 采用列表初始化产出变量个数的集合

In [10]:
print(model.I.data())
print(model.K.data())
print(model.L.data())

(0, 1, 2, 3, 4)
(0, 1, 2)
(0, 1)


In [11]:
model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency')
model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables')

In [12]:
def obj_rule(model):
    return model.theta
    #return model.theta*1  + sum(model.lamda[i] *0 for i in model.I)
model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function')

In [13]:
data=np.array([[ 5,  6,  7, 23, 67],
               [ 5,  9,  7, 14, 34],
               [ 8,  6, 87, 45, 12],
               [34, 67, 32, 11, 37],
               [ 9,  6, 19, 31, 29]])
x=data[0,0:3]
y=data[0,3:5]
xref=data[:,0:3]
yref=data[:,3:5]
def input_rule(model,k):
	"""Return the proper input constraint"""
	return sum(model.lamda[i]*xref[i,k] for i in model.I) <= x[k]
def output_rule(model,l):
    """Return the proper output constraint"""
    return sum(model.lamda[i]*yref[i,l] for i in model.I) >=model.theta*y[l]
model.input = Constraint(model.K, rule=input_rule, doc='input constraint')
model.output = Constraint(model.L, rule=output_rule, doc='output constraint')

In [14]:
model2 = ConcreteModel()
model2.I = Set(initialize = range(4))
model2.J = Set(initialize = range(5))
model2.x = Var(model2.I, model2.J, within = Reals)
model2.c1 = Constraint(expr=2.0 <= model2.x[0,0]+4*model2.x[0,2]) # 不等式约束 2 <= x[0,0] + 4*x[0,2]
model2.c2 = Constraint(expr=3*model2.x[2,3]+5*model2.x[3,4]==10.0) # 等式约束 3*x[2,3] + 5*x[3,4] = 10

In [15]:
## 多维约束为∑j,j≠1xij≤2,i=1,2,3,4\sum_{j,j \neq 1 } x_{ij} \leq 2, i=1,2,3,4 的定义示例，不需要执行
# def c3_rule(model2, i):
#     if j==1:
#         return Objective.Skip
#     return sum([model2.p[i,j]*model2.x[i,j] for j in model2.J]) <= 2
# model2.c3= Constraint(model2.I, rule = c3_rule)

In [16]:
opt = SolverFactory('glpk') # 指定 glpk 作为求解器
solution = opt.solve(model) # 调用求解器求解

solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
opt = SolverFactory('glpk') # 指定 glpk 作为求解器
solution = opt.solve(model,report_timing=True) # 调用求解器求解

        0.00 seconds required to write file
        0.00 seconds required for presolve
        0.01 seconds required for solver
        0.00 seconds required to read logfile 
        0.00 seconds required to read solution file 
        0.00 seconds required for postsolve


In [ ]:
opt = SolverFactory('mosek') # 指定 mosek 作为求解器
solution = opt.solve(model,report_timing=True) # 调用求解器求解

opt = SolverFactory('ipopt') # 指定 ipopt 作为求解器
solution = opt.solve(model,report_timing=True) # 调用求解器求解

# opt = SolverFactory('scip') # 指定 scip 作为求解器
# solution = opt.solve(model,report_timing=True) # 调用求解器求解

opt = SolverFactory('mosek') # 指定 mosek 作为求解器
solution = opt.solve(model,report_timing=True) # 调用求解器求解

        0.00 seconds required for presolve
        0.01 seconds required for solver
        0.00 seconds required for postsolve
        0.00 seconds required to write file
        0.00 seconds required for presolve


        0.03 seconds required for solver
        0.00 seconds required to read logfile 
        0.00 seconds required to read solution file
        0.00 seconds required for postsolve
        0.00 seconds required for presolve
        0.01 seconds required for solver
        0.00 seconds required for postsolve


In [ ]:
theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
obj = value(model.obj) # 提取目标函数 
print("optimum theta: \n {} ".format(theta))
print("optimum lamda: \n {} ".format(lamda))
print("optimal objective: {}".format(obj))

optimum theta: 
 [1.] 
optimum lamda: 
 [1. 0. 0. 0. 0.] 
optimal objective: 1.0


In [ ]:
from pyomo.environ import *
import numpy as np
data=np.array([[ 5,  6,  7, 23, 67],
               [ 5,  9,  7, 14, 34],
               [ 8,  6, 87, 45, 12],
               [34, 67, 32, 11, 37],
               [ 9,  6, 19, 31, 29]])
x=data[0,0:3]
y=data[0,3:5]
xref=data[:,0:3]
yref=data[:,3:5]
model = ConcreteModel()
model.I = Set(initialize = range(5)) # 采用列表初始化技术参照决策单元个数的集合
model.K = Set(initialize = range(3)) # 采用列表初始化投入变量个数的集合
model.L = Set(initialize = range(2)) # 采用列表初始化产出变量个数的集合
model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
def obj_rule(model):
    return model.theta 
    #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
def input_rule(model,k):
    """Return the proper input constraint"""
    return sum(model.lamda[i]*xref[i,k] for i in model.I) <= x[k]
def output_rule(model,l):
    """Return the proper output constraint"""
    return sum(model.lamda[i]*yref[i,l] for i in model.I) >=model.theta*y[l]
model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
opt = SolverFactory('mosek') # 指定 mosek 作为求解器
solution = opt.solve(model) # 调用求解器求解
theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
obj = value(model.obj) # 提取目标函数 
print("optimum theta: \n {} ".format(theta))
print("optimum lamda: \n {} ".format(lamda))
print("optimal objective: {}".format(obj))

optimum theta: 
 [1.] 
optimum lamda: 
 [1. 0. 0. 0. 0.] 
optimal objective: 1.0


In [ ]:
from pyomo.environ import *
import numpy as np
data=np.array([[ 5,  6,  7, 23, 67],
               [ 5,  9,  7, 14, 34],
               [ 8,  6, 87, 45, 12],
               [34, 67, 32, 11, 37],
               [ 9,  6, 19, 31, 29]])
thetalt = np.empty(5)                # 定义thetalt 用于存储计算结果
for j in range(5):              # 在data的行上循环
    x=data[j,0:3]
    y=data[j,3:5]
    xref=data[:,0:3]
    yref=data[:,3:5]
    model = ConcreteModel()
    model.I = Set(initialize = range(5)) # 采用列表初始化技术参照决策单元个数的集合
    model.K = Set(initialize = range(3)) # 采用列表初始化投入变量个数的集合
    model.L = Set(initialize = range(2)) # 采用列表初始化产出变量个数的集合
    model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
    model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
    def obj_rule(model):
        return model.theta 
        #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
    model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
    def input_rule(model,k):
        """Return the proper input constraint"""
        return sum(model.lamda[i]*xref[i,k] for i in model.I) <= x[k]
    def output_rule(model,l):
        """Return the proper output constraint"""
        return sum(model.lamda[i]*yref[i,l] for i in model.I) >=model.theta*y[l]
    model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
    model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
    opt = SolverFactory('mosek') # 指定 mosek 作为求解器
    solution = opt.solve(model) # 调用求解器求解
    theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
    lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
    obj = value(model.obj) # 提取目标函数 
    #print("optimum theta: \n {} ".format(theta))
    #print("optimum lamda: \n {} ".format(lamda))
    #print("optimal objective: {}".format(obj))
    thetalt[j]=theta  
    te = 1/thetalt
print(te)

[1.         0.60869565 1.         0.12080224 1.        ]


C:\Users\abc\AppData\Local\Temp\ipykernel_21140\826372927.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  thetalt[j]=theta
C:\Users\abc\AppData\Local\Temp\ipykernel_21140\826372927.py:41: RuntimeWarning: divide by zero encountered in divide
  te = 1/thetalt


In [ ]:
from pyomo.environ import *
import numpy as np
def dea1(data,dataref,numk):
    """
    data: 待评价决策单元的投入产出数据，按照投入和产出排列
    dataref: 参照技术决策单元的投入产出数据，按照投入和产出排列
    k：表示前k个变量为投入数据
    """
    thetalt = np.empty(data.shape[0])          # 定义thetalt 用于存储计算结果
    for j in range(data.shape[0]):              # 在data的行上循环
        x=data[j,0:numk]
        y=data[j,numk:]
        xref=dataref[:,0:numk]
        yref=dataref[:,numk:]
        model = ConcreteModel()
        model.I = Set(initialize = range(dataref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(numk)) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(data.shape[1]-numk)) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref[i,k] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref[i,l] for i in model.I) >=model.theta*y[l]
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        te = 1/thetalt
    return te

In [ ]:
data=np.array([[ 5,  6,  7, 23, 67],
               [ 5,  9,  7, 14, 34],
               [ 8,  6, 87, 45, 12],
               [34, 67, 32, 11, 37],
               [ 9,  6, 19, 31, 29]])
dataref=data
numk=3
te=dea1(data,dataref,numk)
print(te)

[1.         0.60869565 1.         0.12080224 1.        ]


C:\Users\abc\AppData\Local\Temp\ipykernel_21140\3539517608.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  thetalt[j]=theta


In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np
import warnings
warnings.filterwarnings("ignore")

def dea2(dataframe,varname,numk):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 字符串列表，表示投入和产出变量的列表，如["K","L","Y"]
    k：表示前k个变量为投入数据
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    data = dataframe.loc[:,varname]
    dataref = dataframe.loc[:,varname]
    for j in range(data.shape[0]):              # 在data的行上循环
        x=data.iloc[j,0:numk]
        y=data.iloc[j,numk:]
        xref=dataref.iloc[:,0:numk]
        yref=dataref.iloc[:,numk:]
        xcol=xref.columns
        ycol=yref.columns
        model = ConcreteModel()
        model.I = Set(initialize = range(dataref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(numk)) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = 1/thetadf["theta"]
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea2(ex3,["K","L","Y"],2)
te

,theta,te
0,1.123261,0.890265
1,1.000000,1.000000
2,1.220876,0.819084
3,1.315739,0.760029
4,1.000000,1.000000
5,1.151225,0.868640
6,1.767770,0.565684
7,1.911547,0.523136
8,1.255019,0.796801
9,1.220521,0.819322


In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np
def dea3(dataframe, varname, numk, evaquery):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 字符串列表，表示投入和产出变量的列表，如["K","L","Y"]
    k：表示前k个变量为投入数据
    evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    indexlt = dataframe.query(evaquery).index
    data = dataframe.loc[indexlt,varname]
    dataref = dataframe.loc[indexlt,varname]
    for j in range(data.shape[0]):              # 在data的行上循环
        x=data.iloc[j,0:numk]
        y=data.iloc[j,numk:]
        xref=dataref.iloc[:,0:numk]
        yref=dataref.iloc[:,numk:]
        xcol=xref.columns
        ycol=yref.columns
        model = ConcreteModel()
        model.I = Set(initialize = range(dataref.shape[0])) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(numk)) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(data.shape[1]-numk)) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = 1/thetadf["theta"]
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea3(ex3,["K","L","Y"],2,"dmu==[1,2,3,4,5,6,7,8,9,10]")
te

,theta,te
0,1.123261,0.890265
1,1.000000,1.000000
2,1.220876,0.819084
3,1.315739,0.760029
4,1.000000,1.000000
5,1.151225,0.868640
6,1.767770,0.565684
7,1.911547,0.523136
8,1.255019,0.796801
9,1.220521,0.819322


In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea3(ex3,["K","L","Y"],2,"dmu<=10")
te

,theta,te
0,1.123261,0.890265
1,1.000000,1.000000
2,1.220876,0.819084
3,1.315739,0.760029
4,1.000000,1.000000
5,1.151225,0.868640
6,1.767770,0.565684
7,1.911547,0.523136
8,1.255019,0.796801
9,1.220521,0.819322


In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np
def dea4(dataframe, varname, numk, evaquery, refquery):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 字符串列表，表示投入和产出变量的列表，如["K","L","Y"]
    k：表示前k个变量为投入数据
    evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    indexlt = dataframe.query(evaquery).index
    indexltref = dataframe.query(refquery).index
    data = dataframe.loc[indexlt,varname]
    dataref = dataframe.loc[indexltref,varname]
    for j in range(data.shape[0]):              # 在data的行上循环
        x=data.iloc[j,0:numk]
        y=data.iloc[j,numk:]
        xref=dataref.iloc[:,0:numk]
        yref=dataref.iloc[:,numk:]
        xcol=xref.columns
        ycol=yref.columns
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(numk)) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(data.shape[1]-numk)) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = 1/thetadf["theta"]
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea4(ex3,["K","L","Y"],2,"dmu<=10","dmu<=20")
te

,theta,te
0,1.123261,0.890265
1,1.000000,1.000000
2,1.220876,0.819084
3,1.315739,0.760029
4,1.000000,1.000000
5,1.151225,0.868640
6,1.767770,0.565684
7,1.911547,0.523136
8,1.255019,0.796801
9,1.220521,0.819322


In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea4(ex3,["K","L","Y"],2,"dmu==[10,11,2,3,4,5,6,18]","dmu<=20")
te

,theta,te
0,1.000000,1.000000
1,1.220876,0.819084
2,1.315739,0.760029
3,1.000000,1.000000
4,1.151225,0.868640
5,1.220521,0.819322
6,1.016643,0.983629
7,1.301974,0.768064


In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np
def dea5(dataframe, varname, numk, evaquery, refquery):
    """
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    varname: 字符串列表，表示投入和产出变量的列表，如["K","L","Y"]
    k：表示前k个变量为投入数据
    evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    indexlt = dataframe.query(evaquery).index
    indexltref = dataframe.query(refquery).index
    data = dataframe.loc[indexlt,varname]
    dataref = dataframe.loc[indexltref,varname]
    xcol=varname[0:numk]
    ycol=varname[numk:]
    xref=dataref.loc[:,xcol]
    yref=dataref.loc[:,ycol]
    for j in data.index:              # 在data的索引上循环
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(numk)) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(data.shape[1]-numk)) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = 1/thetadf["theta"]
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea5(ex3,["K","L","Y"],2,"dmu==[10,11,2,3,4,5,6,18]","dmu<=20")
te

,theta,te
1,1.000000,1.000000
2,1.220876,0.819084
3,1.315739,0.760029
4,1.000000,1.000000
5,1.151225,0.868640
9,1.220521,0.819322
10,1.016643,0.983629
17,1.301974,0.768064


In [ ]:
import re
s = "Y ~ K     L  "
inputvars = s.split('~')[1].strip(' ') 
inputvars = re.compile(' +').sub(' ',inputvars).split(' ')
outputvars = s.split('~')[0]    .strip(' ') 
outputvars = re.compile(' +').sub(' ',outputvars).split(' ')

In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def dea6(formula, dataframe, evaquery, refquery):
    """
    formula: 产出变量~投入变量，如“ Y   ~ K     L ”
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    indexlt = dataframe.query(evaquery).index
    indexltref = dataframe.query(refquery).index
    inputvars = formula.split('~')[1].strip(' ') 
    xcol = re.compile(' +').sub(' ',inputvars).split(' ')
    outputvars = formula.split('~')[0].strip(' ') 
    ycol = re.compile(' +').sub(' ',outputvars).split(' ')    
    data = dataframe.loc[indexlt,xcol+ycol]
    dataref = dataframe.loc[indexltref,xcol+ycol]
    xref=dataref.loc[:,xcol]
    yref=dataref.loc[:,ycol]
    for j in data.index:              # 在data的索引上循环
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = 1/thetadf["theta"]
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea6("Y~K L",ex3,"dmu==[10,11,2,3,4,5,6,18]","dmu<=20")
te

,theta,te
1,1.000000,1.000000
2,1.220876,0.819084
3,1.315739,0.760029
4,1.000000,1.000000
5,1.151225,0.868640
9,1.220521,0.819322
10,1.016643,0.983629
17,1.301974,0.768064


In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def dea7(formula, dataframe, rts,evaquery, refquery):
    """
    formula: 产出变量~投入变量，如“ Y   ~ K     L ”
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    rts: 传入表示可变规模报酬或不变规模报酬的字符串，如"crs"，"vrs"
    evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    indexlt = dataframe.query(evaquery).index
    indexltref = dataframe.query(refquery).index
    inputvars = formula.split('~')[1].strip(' ') 
    xcol = re.compile(' +').sub(' ',inputvars).split(' ')
    outputvars = formula.split('~')[0].strip(' ') 
    ycol = re.compile(' +').sub(' ',outputvars).split(' ')    
    data = dataframe.loc[indexlt,xcol+ycol]
    dataref = dataframe.loc[indexltref,xcol+ycol]
    xref=dataref.loc[:,xcol]
    yref=dataref.loc[:,ycol]
    for j in data.index:              # 在data的索引上循环
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense=maximize,doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
        def vrs_rule(model):
            """Return the various return to scale constraint"""
            return sum(model.lamda[i] for i in model.I) == 1
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        if rts == "vrs":
            model.vrs = Constraint(rule=vrs_rule, doc='various return to scale rule') # 定义与可变规模报酬有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = 1/thetadf["theta"]
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea7("Y~K L",ex3,"vrs","dmu==[10,11,2,3,4,5,6,18]","dmu<=20")
te

,theta,te
1,1.000000,1.000000
2,1.159021,0.862797
3,1.057462,0.945660
4,1.000000,1.000000
5,1.094295,0.913830
9,1.214044,0.823693
10,1.000000,1.000000
17,1.148435,0.870750


In [ ]:
from pyomo.environ import *
import pandas as pd ; import numpy as np ; import re
def dea8(formula, dataframe, rts, orient, evaquery=None, refquery=None):
    """
    formula: 产出变量 ~ 投入变量，如“ Y    ~ K     L ”
    data: 待评价决策单元的投入产出数据框，按照投入和产出排列
    rts: 传入表示可变规模报酬或不变规模报酬的字符串，"crs","vrs" 二选一
    orient: 传入表示产出方向或投入方向的字符串，"oo","io" 二选一
    evaquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    refquery:传入数据框.query()方法中的参数，如"dmu==1","dmu==[1,2,3]"
    """
    thetalt = {}          # 定义thetalt 用于存储计算结果
    if type(evaquery)==type(None):
        indexlt = dataframe.index
    else:
        indexlt = dataframe.query(evaquery).index
    if type(refquery)==type(None):
        indexltref = dataframe. index
    else:
        indexltref = dataframe.query(refquery).index
    inputvars = formula.split('~')[1].strip(' ') 
    xcol = re.compile(' +').sub(' ',inputvars).split(' ')
    outputvars = formula.split('~')[0].strip(' ') 
    ycol = re.compile(' +').sub(' ',outputvars).split(' ')    
    data = dataframe.loc[indexlt,xcol+ycol]
    dataref = dataframe.loc[indexltref,xcol+ycol]
    xref=dataref.loc[:,xcol]
    yref=dataref.loc[:,ycol]
    for j in data.index:              # 在data的索引上循环
        x=data.loc[j,xcol]
        y=data.loc[j,ycol]
        model = ConcreteModel()
        model.I = Set(initialize = dataref.index) # 采用列表初始化技术参照决策单元个数的集合
        model.K = Set(initialize = range(len(xcol))) # 采用列表初始化投入变量个数的集合
        model.L = Set(initialize = range(len(ycol))) # 采用列表初始化产出变量个数的集合
        model.theta = Var(within=Reals,bounds=(None, None), doc='efficiency') # 定义决策变量theta
        model.lamda = Var(model.I, bounds=(0.0, None), doc='intensity variables') # 定义决策变量lamda
        def obj_rule(model):
            return model.theta 
            #return model.theta[0]*1  + sum(model.lamda[i] *0 for i in model.I)
        model.obj = Objective(rule = obj_rule, sense= (maximize if orient == "oo" else minimize),doc='objective function') # 定义目标函数
        def input_rule(model,k):
            """Return the proper input constraint"""
            if orient == "oo":
                return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= x[k]
            elif orient == "io":
                return sum(model.lamda[i]*xref.loc[i,xcol[k]] for i in model.I) <= model.theta*x[k]
        def output_rule(model,l):
            """Return the proper output constraint"""
            if orient == "oo":
                return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=model.theta*y[l]
            elif orient == "io":
                return sum(model.lamda[i]*yref.loc[i,ycol[l]] for i in model.I) >=y[l]
        def vrs_rule(model):
            """Return the various return to scale constraint"""
            return sum(model.lamda[i] for i in model.I) == 1
        model.input = Constraint(model.K, rule=input_rule, doc='input constraint') # 定义与投入变量有关的约束条件
        model.output =Constraint(model.L, rule=output_rule, doc='output constraint') # 定义与产出变量有关的约束条件
        if rts == "vrs":
            model.vrs = Constraint(rule=vrs_rule, doc='various return to scale rule') # 定义与可变规模报酬有关的约束条件
        opt = SolverFactory('mosek') # 指定 mosek 作为求解器
        solution = opt.solve(model) # 调用求解器求解
        theta = np.asarray(list(model.theta[:].value)) # 提取决策变量theta
        lamda = np.asarray(list(model.lamda[:].value)) # 提取决策变量lamda
        obj = value(model.obj) # 提取目标函数 
        #print("optimum theta: \n {} ".format(theta))
        #print("optimum lamda: \n {} ".format(lamda))
        #print("optimal objective: {}".format(obj))
        thetalt[j]=theta  
        thetadf =pd.DataFrame(thetalt,index=["theta"]).T
        thetadf["te"] = (1/thetadf["theta"] if orient == "oo" else  thetadf["theta"] )
    return thetadf

In [ ]:
import pandas as pd
ex3 = pd.read_stata(r"../../data/Ex3.dta")
te=dea8("Y~K L",ex3,"vrs","io","dmu==[10,11,2,3,4,5,6,18]","dmu<=20")
te

,theta,te
1,1.000000,1.000000
2,0.874062,0.874062
3,0.960409,0.960409
4,1.000000,1.000000
5,0.925747,0.925747
9,0.825790,0.825790
10,1.000000,1.000000
17,0.889785,0.889785
